# AdaptCMVC - 增量视图聚类

本 Notebook 将 AdaptCMVC 的训练流程转换为分步执行，方便调试和检查。

**优点：**
- ✅ 分步执行，可以随时停止和继续
- ✅ 每步完成后保存检查点，不会丢失进度
- ✅ 可视化中间结果
- ✅ 方便调试和修改参数

## 项目流程

1. **源模型训练** (`source.py`)：训练初始的 VAE 模型
2. **增量视图适配** (`main.py`)：逐步适配新视角
   - View 1：从源模型开始
   - View 2：从 View 1 的最佳模型开始
   - ...


## 配置和初始化

In [3]:
# 导入必要的库
import logging
import numpy as np
import os
import random
import sys

# 在导入 torch 之前禁用 CUDA（如果需要）
USE_CPU = True  # 设置为 False 使用 GPU

if USE_CPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = ''
    os.environ['TORCH_CUDA_ARCH_LIST'] = ''

import torch
import torch.optim as optim

# 导入项目模块
from robustbench.data import load_multiview
from robustbench.sim_utils import clean_accuracy_target as accuracy_target
from robustbench.base_model import ConsistencyAE
import cotta
from data_load import get_val_transformations, get_train_dataset

# 配置日志
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ 库导入成功")


✅ 库导入成功


In [ ]:
# ========================================
# 配置参数（根据你的需求修改）
# ========================================

class Args:
    """参数配置类"""
    def __init__(self):
        # 基本参数
        self.alpha = 0.1
        self.BATCH_SIZE = 16  # CPU 模式使用较小批次
        self.consis = 5
        self.N = 1
        self.sample_num_each_clusters = 15
        self.contra = 0.5
        self.temperature = 0.5
        self.up_alpha = 0.2
        
        # 系统参数
        self.cuda_device = 'cpu' if USE_CPU else '0'
        self.seed = 3407
        self.CROP_SIZE = 64
        self.ADAPTATION = 'cotta'
        self.name = 'coil-100'
        self.root = 'MyData'
        self.views = 3  # 总共 3 个视角
        self.NUM_EX = 1920
        self.class_num = 100
        
        # 优化器参数
        self.STEPS = 1
        self.EPISODIC = False
        self.LR = 0.0001
        self.BETA = 0.9
        self.WD = 0.0
        self.METHOD = 'Adam'

args = Args()

# 设置设备和随机种子
if args.cuda_device.lower() == 'cpu':
    device = torch.device('cpu')
else:
    device = torch.device(f'cuda:{args.cuda_device}' if torch.cuda.is_available() else 'cpu')

print(f"✅ 使用设备: {device}")

# 设置随机种子
def setup_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available() and device.type == 'cuda':
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    if device.type == 'cuda':
        torch.backends.cudnn.deterministic = True

setup_seed(args.seed)
print(f"✅ 随机种子设置为: {args.seed}")


## 辅助函数


In [ ]:
# ========================================
# 辅助函数
# ========================================

def setup_source(model):
    """设置源模型（无适配）"""
    model.eval()
    return model

def setup_optimizer_custom(params):
    """设置优化器"""
    if args.METHOD == 'Adam':
        return optim.Adam(params,
                    lr=args.LR,
                    betas=(args.BETA, 0.999),
                    weight_decay=args.WD)
    elif args.METHOD == 'SGD':
        return optim.SGD(params,
                   lr=args.LR,
                   momentum=0.9,
                   dampening=0,
                   weight_decay=args.WD,
                   nesterov=True)
    else:
        raise NotImplementedError

def setup_cotta_model(model):
    """设置 CoTTA 模型"""
    model = cotta.configure_model(model)
    params, param_names = cotta.collect_params(model)
    optimizer = setup_optimizer_custom(params)
    cotta_model = cotta.CoTTA(model, optimizer,
                           steps=args.STEPS,
                           episodic=args.EPISODIC,
                            num_classes = args.class_num,
                              CROP_SIZE = args.CROP_SIZE,
                              contra=args.contra,
                              consis=args.consis,
                              N=args.N,
                              sample_num_each_clusters = args.sample_num_each_clusters)
    return cotta_model

print("✅ 辅助函数定义完成")


## 步骤1：View 1 训练


In [ ]:
# ========================================
# View 1: 从源模型开始训练
# ========================================

print("=" * 80)
print("开始 View 1 训练")
print("=" * 80)

# 1. 构建模型
AE = ConsistencyAE(
    basic_hidden_dim=32, c_dim=20, continous=True, in_channel=3, 
    num_res_blocks=3, ch_mult=[1, 2, 4, 8], block_size=8, temperature=1.0,
    latent_ch=8, kld_weight=1.0, views=1, categorical_dim=args.class_num
)

# 2. 加载源模型
AE.load_state_dict(
    torch.load('./source_model/v1-best-2806-185-0.5641.pth', map_location='cpu'),
    strict=False
)
base_model = AE.to(device)
print("✅ 已加载源模型")

# 3. 设置适配模型
model = setup_cotta_model(base_model)
print("✅ 已设置 CoTTA 模型")

# 4. 加载数据和先验
val_transformations = get_val_transformations(args.CROP_SIZE)
train_dataset = get_train_dataset(args.name, args.root, args.views, val_transformations)

views = 1
source_result = np.load(f'./source_model/v1-20source_result.npy')
source_result = torch.from_numpy(source_result).to(device)
source_center = np.load(f'./source_model/v1-20source_center.npy')
print("✅ 已加载数据和先验")

print("\n开始训练 View 1...\n")


In [ ]:
# ========================================
# View 1: 训练循环（50次评估）
# ========================================

result_dir = os.path.join("./last_sim_model")
os.makedirs(result_dir, exist_ok=True)

best_loss = np.inf
best_acc = 0.
old_best_model_path = ""
acc = []

for i in range(50):
    # 加载数据
    x_test, y_test = load_multiview(args.NUM_EX, False, train_dataset)
    x_test, y_test = x_test[views].to(device), y_test.to(device)
    
    # 评估和适配
    result, kmeans_pre, r_loss, c_loss, str_loss, cluster_center = accuracy_target(
        source_center, source_result, model, x_test, y_test,
        args.BATCH_SIZE, args.class_num, views, args.up_alpha, device=device
    )
    
    cur_loss = r_loss + c_loss
    acc.append(result['consist-acc'])
    
    print(f"[迭代 {i+1}/50] ", ', '.join([f'{k}:{v:.4f}' for k, v in result.items()]))
    
    # 保存最佳模型
    if cur_loss <= best_loss:
        best_loss = cur_loss.item()
        best_acc = result['consist-acc']
        best_model_path = os.path.join(result_dir, f'last_sim_model--{int(views)}.pth')
        if old_best_model_path and os.path.exists(old_best_model_path):
            os.remove(old_best_model_path)
        old_best_model_path = best_model_path
        
        np.save(f'./last_sim_model/last_sim_result.npy', kmeans_pre)
        np.save(f'./last_sim_model/last_sim_center.npy', cluster_center)
        model.eval()
        torch.save(model.state_dict(), best_model_path)
        print(f"  💾 保存最佳模型: acc={best_acc:.4f}, loss={best_loss:.4f}")

print(f"\n✅ View 1 训练完成！最佳准确率: {best_acc:.4f}")
np.save(f'./last_sim_model/acc-{int(views)}.npy', np.array(acc))


## 步骤2：View 2 训练


In [ ]:
# ========================================
# View 2: 从 View 1 的最佳模型继续训练
# ========================================

print("=" * 80)
print("开始 View 2 训练")
print("=" * 80)

# 1. 重新构建模型
AE = ConsistencyAE(
    basic_hidden_dim=32, c_dim=20, continous=True, in_channel=3, 
    num_res_blocks=3, ch_mult=[1, 2, 4, 8], block_size=8, temperature=1.0,
    latent_ch=8, kld_weight=1.0, views=1, categorical_dim=args.class_num
)

# 2. 加载 View 1 的最佳模型
AE.load_state_dict(
    torch.load(f'./last_sim_model/last_sim_model--{1}.pth', map_location='cpu'),
    strict=False
)
base_model = AE.to(device)
print("✅ 已加载 View 1 的最佳模型")

# 3. 设置适配模型
model = setup_cotta_model(base_model)
print("✅ 已设置 CoTTA 模型")

# 4. 加载 View 1 的聚类结果
views = 2
source_result = np.load(f'./last_sim_model/last_sim_result.npy')
source_result = torch.from_numpy(source_result).to(device)
source_center = np.load(f'./last_sim_model/last_sim_center.npy')
print("✅ 已加载 View 1 的聚类先验")

print("\n开始训练 View 2...\n")


In [ ]:
# ========================================
# View 2: 训练循环（50次评估）
# ========================================

best_loss = np.inf
best_acc = 0.
old_best_model_path = ""
acc_view2 = []

for i in range(50):
    # 加载数据
    x_test, y_test = load_multiview(args.NUM_EX, False, train_dataset)
    x_test, y_test = x_test[views].to(device), y_test.to(device)
    
    # 评估和适配
    result, kmeans_pre, r_loss, c_loss, str_loss, cluster_center = accuracy_target(
        source_center, source_result, model, x_test, y_test,
        args.BATCH_SIZE, args.class_num, views, args.up_alpha, device=device
    )
    
    cur_loss = r_loss + c_loss
    acc_view2.append(result['consist-acc'])
    
    print(f"[迭代 {i+1}/50] ", ', '.join([f'{k}:{v:.4f}' for k, v in result.items()]))
    
    # 保存最佳模型
    if cur_loss <= best_loss:
        best_loss = cur_loss.item()
        best_acc = result['consist-acc']
        best_model_path = os.path.join(result_dir, f'last_sim_model--{int(views)}.pth')
        if old_best_model_path and os.path.exists(old_best_model_path):
            os.remove(old_best_model_path)
        old_best_model_path = best_model_path
        
        np.save(f'./last_sim_model/last_sim_result.npy', kmeans_pre)
        np.save(f'./last_sim_model/last_sim_center.npy', cluster_center)
        model.eval()
        torch.save(model.state_dict(), best_model_path)
        print(f"  💾 保存最佳模型: acc={best_acc:.4f}, loss={best_loss:.4f}")

print(f"\n✅ View 2 训练完成！最佳准确率: {best_acc:.4f}")
np.save(f'./last_sim_model/acc-{int(views)}.npy', np.array(acc_view2))


## 训练结果总结


In [ ]:
# ========================================
# 加载并显示训练结果
# ========================================

import matplotlib.pyplot as plt

# 加载准确率历史
acc_view1 = np.load('./last_sim_model/acc-1.npy')
acc_view2 = np.load('./last_sim_model/acc-2.npy')

# 绘制准确率曲线
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(acc_view1, label='View 1')
plt.xlabel('迭代次数')
plt.ylabel('准确率')
plt.title('View 1 训练曲线')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(acc_view2, label='View 2')
plt.xlabel('迭代次数')
plt.ylabel('准确率')
plt.title('View 2 训练曲线')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# 打印最终结果
print("\n" + "=" * 80)
print("🎉 训练完成！")
print("=" * 80)
print(f"View 1 最佳准确率: {acc_view1.max():.4f}")
print(f"View 2 最佳准确率: {acc_view2.max():.4f}")
print("=" * 80)
